## Week 3 Assignment
### Segmenting and Clustering Neighborhoods in Toronto
#### Please kindly review

In [1]:
# get HTML file and scraping with BeautifulSoup
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})

In [2]:
# Assign values to lists
A=[]
B=[]
C=[]

for row in My_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [3]:
# Create dataframe
import pandas as pd
df=pd.DataFrame()
df['PostalCode']=A
df['Borough']=B
df['Neighborhood']=C

In [4]:
# drop rows with 'Not assigned' value in Borough column
df1=df[(df['Borough']!= 'Not assigned')]

In [5]:
# Join columns of same PostalCode and Borough
df2=df1.groupby(['PostalCode','Borough'], sort = False).agg(lambda x: ', '.join(x))
df2.reset_index(inplace = True)

In [6]:
# replace Not assigned Neighborhoods with Borough
df2['Neighborhood']=df2['Neighborhood'].replace('Not assigned\n', df2['Borough'])

In [7]:
# sort dataframe by Postalcode
df2.sort_values(by='PostalCode', ascending=True, axis=0, inplace=True)

In [8]:
# read in Geospatial dataset
import pandas as pd
geo = pd.read_csv("http://cocl.us/Geospatial_data")

In [9]:
df2['Latitude']=geo['Latitude']

In [10]:
df2['Longitude']=geo['Longitude']

In [11]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
!conda install -c conda-forge folium #import folium

Solving environment: | ^C
failed

CondaError: KeyboardInterrupt



In [12]:
df2.head(2)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
6,M1B,Scarborough,"Rouge, Malvern",43.727929,-79.262029
12,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.794200,-79.262029


In [19]:
import folium as folium
# define the world map
toronto_map = folium.Map()
# define the world map centered around Canada with a low zoom level
toronto_map = folium.Map(location=[43.7, -79.3], zoom_start=10)

In [24]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
from folium import plugins
# instantiate a feature group for the incidents in the dataframe
Neighborhoods = folium.map.FeatureGroup()
# instantiate a mark cluster object for the Neighborhoods in the dataframe
Neighborhoods = plugins.MarkerCluster().add_to(toronto_map)
# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(df2['Latitude'], df2['Longitude'], df2['Neighborhood']):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(Neighborhoods)
# display map
toronto_map